In [1]:
import os
import pandas as pd
import spacy 

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/hanzhang0420/Women-Clothing-E-commerce/master/Womens%20Clothing%20E-Commerce%20Reviews.csv")

In [3]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [4]:
df.isna().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [5]:
df.shape

(23486, 11)

In [6]:
df = df[['Review Text','Recommended IND']].dropna()

In [7]:
df.head(6)

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
5,"I love tracy reese dresses, but this one is no...",0


In [8]:
# Negative Text 
df.iloc[5]['Review Text']

'I love tracy reese dresses, but this one is not for the very petite. i am just under 5 feet tall and usually wear a 0p in this brand. this dress was very pretty out of the package but its a lot of dress. the skirt is long and very full so it overwhelmed my small frame. not a stranger to alterations, shortening and narrowing the skirt would take away from the embellishment of the garment. i love the color and the idea of the style but it just did not work on me. i returned this dress.'

In [9]:
nlp=spacy.load("en_core_web_sm")
nlp.pipe_names

['tagger', 'parser', 'ner']

In [10]:
# we created a simple Text cat we are adding that to spacy :D 
textcat = nlp.create_pipe( "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})

In [11]:
# Adding this to Pipe
nlp.add_pipe(textcat, last=True)

In [12]:
nlp.pipe_names

['tagger', 'parser', 'ner', 'textcat']

In [13]:
# Adding the labels to textcat
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")


1

In [14]:
textcat.labels

('POSITIVE', 'NEGATIVE')

In [15]:
# Converting review text to tuple 
df['tuples'] = df.apply(lambda row: (row['Review Text'],row['Recommended IND']), axis=1)

In [17]:
df.head()

,Review Text,Recommended IND,tuples
0,Absolutely wonderful - silky and sexy and comf...,1,(Absolutely wonderful - silky and sexy and com...
1,Love this dress! it's sooo pretty. i happene...,1,(Love this dress! it's sooo pretty. i happen...
2,I had such high hopes for this dress and reall...,0,(I had such high hopes for this dress and real...
3,"I love, love, love this jumpsuit. it's fun, fl...",1,"(I love, love, love this jumpsuit. it's fun, f..."
4,This shirt is very flattering to all due to th...,1,(This shirt is very flattering to all due to t...


In [18]:
# This is how data looks like 
df["tuples"][0]

('Absolutely wonderful - silky and sexy and comfortable', 1)

In [19]:
# Converting tuple to List 
train = df['tuples'].tolist()

In [20]:
print(train[0])
print(len(train))

('Absolutely wonderful - silky and sexy and comfortable', 1)
22641


In [21]:
texts, labels = zip(*train)

In [22]:
texts[0]

'Absolutely wonderful - silky and sexy and comfortable'

In [23]:
labels[0]

1

In [24]:
cats = []
for y in labels:
    if(bool(y)):
        cats.append({"POSITIVE": True, "NEGATIVE":False})
    else:
        cats.append({"POSITIVE": False, "NEGATIVE":True})

In [25]:
TrainX = texts
TrainY = cats

In [26]:
n_texts=23486


In [27]:
len(TrainX)

22641

In [28]:
len(TrainY)

22641

In [29]:
train_data = list(zip(TrainX,[{'cats': cats} for cats in TrainY]))

In [30]:
# Data has no formate
train_data[0]

('Absolutely wonderful - silky and sexy and comfortable',
 {'cats': {'POSITIVE': True, 'NEGATIVE': False}})

In [31]:
len(train_data)


22641

In [32]:
n_iter=10

In [33]:
from spacy.util import minibatch, compounding

# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    
    print("Training the model...")
    
    # Performing training
    for i in range(n_iter):
        print("Epoch : {} ".format(i))
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)

Training the model...
Epoch : 0 


KeyboardInterrupt: 

In [ ]:
nlp.to_disk("sentiment")

In [ ]:
nlp = spacy.load("sentiment")

In [ ]:
test_text = "I had such high hopes for this dress and really crappy worst product hate it wporst bad "
doc=nlp(test_text)
doc.cats 